# Verificando o conteúdo do dataset

> ação:
> 1. Verificar necessidade de limpeza dos dados (texo).  a data precisaremos separar o horario
> 2. Verificar qual coluna do texto vamos utilizar
> 3. Estudar a polaridade das palavras
> 4. Verificar palavras mais utilizadas
> 5. Buscar dados econômicos e o price action do EURUSD
> 6. Cenas dos próximos insights

## Começou!

In [1]:
#import das bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('C:\Python\Mentoria\Forex\Forex\Dataset\compilado\cnbc_dataset.csv')
df.head(3)

,title,published_at,publisher,short_description,description
0,Santoli’s Wednesday market notes: Could Septem...,2021-09-29 17:09:39+00:00,CNBC,"This is the daily notebook of Mike Santoli, CN...","This is the daily notebook of Mike Santoli, CN..."
1,My take on the early Brexit winners and losers,2016-06-24 13:50:48-04:00,CNBC,This commentary originally ran on Facebook. Bo...,NaN
2,Europe&#039;s recovery depends on Renzi&#039;s...,2014-03-25 13:29:45-04:00,CNBC,"In spring, ambitious reforms began in Italy. U...",NaN


## Cuidando da coluna published_at

In [3]:
#tirando "+00:00" da coluna e transformando em datetime do pandas
df['published_at'] = [x.replace('+00:00', "") for x in df['published_at']]
df['published_at'] = [x.replace('-04:00', "") for x in df['published_at']]
df['published_at'] = pd.to_datetime(df['published_at'])

df.head(3)

,title,published_at,publisher,short_description,description
0,Santoli’s Wednesday market notes: Could Septem...,2021-09-29 17:09:39,CNBC,"This is the daily notebook of Mike Santoli, CN...","This is the daily notebook of Mike Santoli, CN..."
1,My take on the early Brexit winners and losers,2016-06-24 13:50:48,CNBC,This commentary originally ran on Facebook. Bo...,NaN
2,Europe&#039;s recovery depends on Renzi&#039;s...,2014-03-25 13:29:45,CNBC,"In spring, ambitious reforms began in Italy. U...",NaN


In [4]:
df['new_date'] = [d.date() for d in df['published_at']]
df['new_time'] = [d.time() for d in df['published_at']]
df.head(3)

,title,published_at,publisher,short_description,description,new_date,new_time
0,Santoli’s Wednesday market notes: Could Septem...,2021-09-29 17:09:39,CNBC,"This is the daily notebook of Mike Santoli, CN...","This is the daily notebook of Mike Santoli, CN...",2021-09-29,17:09:39
1,My take on the early Brexit winners and losers,2016-06-24 13:50:48,CNBC,This commentary originally ran on Facebook. Bo...,NaN,2016-06-24,13:50:48
2,Europe&#039;s recovery depends on Renzi&#039;s...,2014-03-25 13:29:45,CNBC,"In spring, ambitious reforms began in Italy. U...",NaN,2014-03-25,13:29:45


In [5]:
df['new_date'] = pd.to_datetime(df['new_date'])

#df['new_time'] = pd.to_datetime(df['new_time'])

In [6]:
df['new_date'].min(), df['new_date'].max()

(Timestamp('2006-12-04 00:00:00'), Timestamp('2021-10-18 00:00:00'))

## Verificando a polaridade da coluna 'title' utilizando o textblob

TextBlob aims to provide access to common text-processing operations through a familiar interface. You can treat TextBlob objects as if they were Python strings that learned how to do Natural Language Processing.

https://textblob.readthedocs.io/en/dev/quickstart.html

### Sentiment Analysis
The sentiment property returns a namedtuple of the form Sentiment(polarity, subjectivity). 

The polarity score is a float within the range [-1.0, 1.0]. 

The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.

>>> testimonial = TextBlob("Textblob is amazingly simple to use. What great fun!")

>>> testimonial.sentiment

Sentiment(polarity=0.39166666666666666, subjectivity=0.4357142857142857)

>>> testimonial.sentiment.polarity

0.39166666666666666

In [7]:
#!pip install textblob


In [8]:
#!pip install --upgrade pip

In [9]:
from textblob import TextBlob, Word


In [10]:
df_sentimento = df.copy()
df_sentimento.head(2)

,title,published_at,publisher,short_description,description,new_date,new_time
0,Santoli’s Wednesday market notes: Could Septem...,2021-09-29 17:09:39,CNBC,"This is the daily notebook of Mike Santoli, CN...","This is the daily notebook of Mike Santoli, CN...",2021-09-29,17:09:39
1,My take on the early Brexit winners and losers,2016-06-24 13:50:48,CNBC,This commentary originally ran on Facebook. Bo...,NaN,2016-06-24,13:50:48


In [11]:
df_sentimento['sentiment'] = df_sentimento['title'].apply(lambda x: TextBlob(x).sentiment_assessments)
df_sentimento.head(2)

,title,published_at,publisher,short_description,description,new_date,new_time,sentiment
0,Santoli’s Wednesday market notes: Could Septem...,2021-09-29 17:09:39,CNBC,"This is the daily notebook of Mike Santoli, CN...","This is the daily notebook of Mike Santoli, CN...",2021-09-29,17:09:39,"(0.0, 0.0, [(['fourth'], 0.0, 0.0, None)])"
1,My take on the early Brexit winners and losers,2016-06-24 13:50:48,CNBC,This commentary originally ran on Facebook. Bo...,NaN,2016-06-24,13:50:48,"(-0.05, 0.25, [(['early'], 0.1, 0.3, None), ([..."


In [12]:
#podemos fazer assim tb: (obrigada, StackOverFlow!)
"""def sentiment_calc(text):
    try:
        return TextBlob(text).sentiment_assessments
    except:
        return None

df['sentiment'] = df['title'].apply(sentiment_calc)
"""


"def sentiment_calc(text):\n    try:\n        return TextBlob(text).sentiment_assessments\n    except:\n        return None\n\ndf['sentiment'] = df['title'].apply(sentiment_calc)\n"

In [40]:
sentiment = list(df_sentimento.sentiment)
sentiment = pd.DataFrame(sentiment)
sentiment.head(3)

In [45]:
assessments = list(sentiment.assessments)
assessments = pd.DataFrame(assessments)
assessments.head(2)

,0,1,2,3,4
0,"([fourth], 0.0, 0.0, None)",None,None,None,None
1,"([early], 0.1, 0.3, None)","([losers], -0.2, 0.2, None)",None,None,None


In [49]:
df_sentimento = pd.concat([df_sentimento, sentiment], axis=1)
df_sentimento.columns

Index(['title', 'published_at', 'publisher', 'short_description',
       'description', 'new_date', 'new_time', 'sentiment', 'polarity',
       'subjectivity', 'assessments'],
      dtype='object')

In [51]:
df_sentimento.to_csv('cnbc_polaridade.csv', index = False)